In [1]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check
import re
import duckdb
import pandas as pd

# Caminho para o banco de dados DuckDB
db_path = "D:\\GitHub\\projeto_Analytics_Engineering\\data_quality\\bronze.duckdb"

In [2]:
# Função para validar o formato monetário como uma string
def check_price_format(val):
    return bool(re.match(r"^\$\d+(,\d{3})*(\.\d{2})?$", val))

# Definindo o esquema para a tabela "listings"
listings_schema = DataFrameSchema({
    "id": Column(pa.Int64, checks=Check.greater_than(0), nullable=False),  # ID não nulo e maior que 0
    "neighbourhood_cleansed": Column(pa.String, nullable=False),           # Bairro não nulo
    #"price": Column(pa.String, checks=Check(check_price_format, element_wise=True), nullable=False),  # Preço obrigatório, formato monetário
    "number_of_reviews": Column(pa.Int64, checks=Check.greater_than_or_equal_to(0), nullable=False),   # Número de avaliações, não pode ser nulo, mínimo de 0
    #"review_scores_rating": Column(pa.Float64, checks=Check.in_range(0, 5), nullable=False)          # Pontuação entre 0 e 5, não pode ser nulo
})


d:\Python\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
# Conectando ao banco de dados DuckDB
conn = duckdb.connect(database=db_path, read_only=False)

# Carregando os dados da tabela "listings"
df_listings = conn.execute("SELECT * FROM listings").fetchdf()

# Validando o DataFrame com Pandera
try:
    validated_listings_df = listings_schema.validate(df_listings)
    print("Dados validados com sucesso:")
    print(validated_listings_df)
except pa.errors.SchemaError as e:
    print("Erro na validação dos dados:", e)
finally:
    conn.close()  # Fechando a conexão com o banco de dados

Dados validados com sucesso:
                        id                                       listing_url  \
0                    17878                https://www.airbnb.com/rooms/17878   
1                    25026                https://www.airbnb.com/rooms/25026   
2                   220377               https://www.airbnb.com/rooms/220377   
3                    35764                https://www.airbnb.com/rooms/35764   
4                   223073               https://www.airbnb.com/rooms/223073   
...                    ...                                               ...   
69323  1185514583461005119  https://www.airbnb.com/rooms/1185514583461005119   
69324  1185571519517844533  https://www.airbnb.com/rooms/1185571519517844533   
69325  1185607754513758247  https://www.airbnb.com/rooms/1185607754513758247   
69326  1185618650054465875  https://www.airbnb.com/rooms/1185618650054465875   
69327  1185684485743879215  https://www.airbnb.com/rooms/1185684485743879215   

          

## Resumo:

coluna price: 1944 nulos

coluna review_scores_rating: 16892 nulos